Econometría Aplicada. Lección 5
===============================

**Author:** Marcos Bujosa



<div class="abstract" id="orgd80ee3e">
<p>
Esta lección veremos las dificultades que ocasiona la correlación
serial y algunos tipos de procesos débilmente estacionarios que nos
permitirán lidiar con ella.
</p>

</div>

-   [lección en html](https://mbujosab.github.io/EconometriaAplicada-SRC/Lecc05.html)
-   [lección en mybinder](https://mybinder.org/v2/gh/mbujosab/EconometriaAplicada-SRC/HEAD?labpath=Lecc05.ipynb)



#### Carga de algunas librerías de R



Primero cargamos la librería `tfarima` (Repositorio Cran:
[https://cran.r-project.org/web/packages/tfarima/index.html](https://cran.r-project.org/web/packages/tfarima/index.html);
repositorio GitHub: [https://github.com/gallegoj/tfarima](https://github.com/gallegoj/tfarima))



In [1]:
library(tfarima)      # librería de José Luis Gallego para Time Series
library(readr)        # para leer ficheros CSV
library(ggplot2)      # para el scatterplot (alternaticamente library(tidyverse))
library(ggfortify)    # para pintar series temporales
library(jtools)       # para representación resultados estimación
library(zoo)          # para generar objetos ts (time series)

y además fijamos los parámetros por defecto para las figuras en `png`
del notebook



In [1]:
# fijamos el tamaño de las figuras que se generan en el notebook
options(repr.plot.width = 12, repr.plot.height = 4, repr.plot.res = 200)

## Series temporales vs datos de sección cruzada



Corresponden a observaciones de un mismo objeto a lo largo del
tiempo. El índice indica el instante de cada medición. *El orden
cronológico puede ser crucial* al modelar los datos.

-   El motivo es que frecuentemente el valor medido en un instante de
    tiempo está relacionado con otras mediciones próximas en el tiempo
    (*correlación serial*).

-   Si es así, ya no deberíamos asumir que las variables aleatorias del
    proceso estocástico subyacente, $\boldsymbol{X}=(X_t\mid
      t\in\mathbb{Z})$, son independientes entre sí.

Esto tiene importantes implicaciones en las técnicas de análisis y
los modelos a utilizar.



##### Población en Australia



In [1]:
PoblacionAustralia_ts = as.ts( read.zoo('datos/PoblacionAustralia.csv', 
                                        header=TRUE,
                                        index.column = 1, 
                                        sep=",", 
                                        FUN = as.yearmon))
p <- autoplot(PoblacionAustralia_ts)
p <- p + labs(y = "Habitantes", x = "Años") + ggtitle("Población australiana (datos anuales)")
p <- p + scale_x_continuous(breaks = scales::pretty_breaks(n = 20))
p

![img](./img/lecc05/PoblacionAustralia.png)



##### PIB UEM



In [1]:
PIB_UEM_df <- read_csv("datos/PIB_UEM.csv",
                     show_col_types = FALSE)
fmt <- "%YQ%q"
PIB_UEM_df$Time <- as.yearqtr(PIB_UEM_df$obs, format = fmt)
# head(PIB_UEM_df,3)
P <- ggplot(PIB_UEM_df, aes(Time, PIB))
P <- P + geom_point() + geom_line()
P <- P + scale_x_continuous(breaks = scales::pretty_breaks(n = 15))
P <- P + labs(y = "Miles de millones de euros", x = "Años") + ggtitle("PIB UEM a precios corrientes (datos trimestrales). Fuente Banco de España")
P

![img](./img/lecc05/PIB_UEM.png)



##### Temperatura media en el Parque del Retiro. Madrid



In [1]:
TemperaturaRetiro_df <- read_csv("datos/Retiro.txt", show_col_types = FALSE)
# Añadimos fechas
TemperaturaRetiro_df$Time <- as.yearmon(1985 + seq(0, nrow(TemperaturaRetiro_df)-1)/12)

P <- ggplot(TemperaturaRetiro_df, aes(Time, TemperaturaMedia))
P <- P + geom_line() # + geom_point() 
P <- P + scale_x_continuous(breaks = scales::pretty_breaks(n = 25))
P <- P + labs(y = "Grados Celsius", x = "Años") + ggtitle("Temperatura media mensual en el Parque del Retiro. Fuente: Comunidad de Madrid")
P

![img](./img/lecc05/TemperaturaReriro.png)



##### Rendimiento porcentual diario del IBEX 35 (std)



In [1]:
IBEX35_ts = as.ts( read.csv.zoo("datos/IBEX35.csv", 
                                strip.white = TRUE))
P <- autoplot(IBEX35_ts) + scale_y_continuous(breaks = scales::pretty_breaks(n = 12))
p <- P + labs(y = "Desviaciones tipicas", x = "Días") + ggtitle("Rendimiento porcentual diario del IBEX 35 (std.). Fuente: Archivo Prof. Miguel Jerez")
p

![img](./img/lecc05/IBEX35.png)

-   Datos centrados y estandarizados, i.e. el eje vertical está en desviaciones típicas.
-   Los *volatility clustering* son característicos de series financieras de alta frecuencia.



##### Producción de cemento



In [1]:
ProduccionCemento_df <- read_csv("datos/ProduccionCemento.csv",
                     show_col_types = FALSE)
fmt <- "%YM%m"
ProduccionCemento_df$Time <- as.yearmon(ProduccionCemento_df$obs, format = fmt)
# head(ProduccionCemento_df,3)
P <- ggplot(ProduccionCemento_df, aes(Time, ProduccionCemento))
P <- P + geom_line() # + geom_point() 
P <- P + scale_x_continuous(breaks = scales::pretty_breaks(n = 25))
P <- P + labs(y = "Miles de Toneladas métricas", x = "Años") + ggtitle("Produccion de cemento (Datos mensuales). Fuente Banco de España")
P

![img](./img/lecc05/ProduccionCemento.png)



### Correlación serial vs muestreo aleatorio simple



Con datos de

-   **sección cruzada:** solemos asumir que el muestreo es aleatorio
    simple
    -   i.e., los datos son realizaciones de variables aleatorias i.i.d.

-   **series temporales:** dicha asunción resulta generalmente errónea
    
    -   con frecuencia el nivel esperado (o la volatilidad) parece cambiar con $t$
    -   con frecuencia hay dependencia temporal (correlación serial).
    
    **Ejemplo**: no parece aceptable asumir que $ProdCemento_{1960M01}$ se
    distribuye igual que $ProdCemento_{2000M04}$ (ni que sea
    independiente de $ProdCemento_{1959M01}$).

Veamos por qué esto genera dificultades&#x2026;



Consideremos el proceso estocástico $$\boldsymbol{X}=(X_t \mid
t=0,\pm1,\pm2,\ldots).$$ Caracterizar su distribución conjunta (todos
los momentos) es demasiado ambicioso.



Así que, tentativamente, vamos a fijarnos *solo* en los dos primeros
momentos:

$$E(X_t)={\color{blue}{ \mu_t}}\quad\text{ y }\quad
Cov(X_t,X_k)=E\big[(X_t-\mu_t)(X_k-\mu_k)\big]={\color{blue}{\lambda_{t,k}}};\quad t,k\in\mathbb{Z}$$

(si $\;k=t\;$ entonces $\;\lambda_{t,t}=Var(X_t)=\sigma^2_t$).

Si el proceso $\boldsymbol{X}$ fuera gaussiano, conocer estos
*parámetros* bastaría para caracterizar la distribución conjunta. Pero
aún así&#x2026;



-   necesitaríamos para cada $X_t$ una muestra suficiente para estimar los parámetros 
    -   pero en una serie temporal tenemos una sola realización de cada $X_t$.

-   Además&#x2026; para cada variable aleatoria $X_t$ hay infinitos parámetros.



### Tipos de procesos estocásticos que simplifican el escenario



-   Si es [débilmente estacionario](./Lecc01.slides.html#/3/1) se reduce drásticamente el número de
    parámetros:
    
    \begin{eqnarray}
    E(X_t)  & = \mu \\
    Cov(X_t,X_{t-k}) & = \gamma_k
    \end{eqnarray}
-   Si además es i.i.d. podemos interpretar la serie temporal como una
    realización de un muestreo aleatorio simple.



El desafío para el analista es (y nótese el abuso de lenguaje)

-   **primero:** transformar los datos para lograr que sean "***estacionarios***".
    -   (Algo vimos en la lección 1))
-   **después:** transformar los datos estacionarios en una secuencia de
    "**datos *i.i.d***" 
    -   (Aún no lo hemos visto)

Todo este proceso constituye la especificación y ajuste de un modelo a
la serie temporal.



Antes de atacar los temas de especificación y ajuste de modelos,
debemos estudiar un poco los procesos estocásticos débilmente
estacionarios que vamos a utilizar.



## Procesos estocásticos de segundo orden



El ambiente natural para estudiar las propiedades de segundo orden de
una colección de variables aleatorias es el espacio de variables
aleatorias $X$ definidas en un espacio de probabilidad tales que
$$E(X)=0 \quad\text{y}\quad E(X^2)<\infty$$ donde $E$ es el operador
esperanza. Denotaremos este espacio con $H$.



### Un poco de geometría



El espacio, dotado de producto escalar y norma $$\langle X \mid Y
\rangle=E(XY),\qquad \lVert X \rVert= \sqrt{E(X^2)},\qquad X,Y \in
H,$$ es un espacio de Hilbert,

Nótese que como las variables de $H$ tienen esperanza cero, el
producto escalar entre $X,Y\in H$ también es $$\langle X \mid Y
\rangle=Cov(X,Y).$$ Por tanto, en este espacio $H$ la noción
geométrica de ortogonalidad coincide con la noción estadística de *no
correlación*. Por tanto, en este contexto los términos producto
escalar y covarianza serán intercambiables.



Una colección de variables aleatorias pertenecientes a $H$
$$\boldsymbol{X}=(X_t\mid t\in\mathbb{Z}) \;\text{ con }\; X_t\in H$$
se denomina *proceso estocástico de segundo orden*.



### Función de covarianzas



La *función de covarianzas* de un proceso estocástico $\boldsymbol{X}$
segundo orden es $$\boldsymbol{\gamma}=(\gamma_{s,t}\mid
s,t\in\mathbb{Z})$$ donde $\gamma_{s,t}=Cov(X_s,X_t)\quad
s,t\in\mathbb{Z}$.

Así, para cada par $(s,t)$, la covarianza $\gamma_{s,t}$ mide la
dependencia lineal entre $X_s$ y $X_t$.



Esta función $\boldsymbol{\gamma}$ es demasiado general, por eso nos
restringiremos a la subclase de procesos estocásticos *débilmente
estacionarios*; pues simplifican enormemente la *función de
covarianzas* $\boldsymbol{\gamma}$.



### Proceso estocástico (débilmente) estacionario y su ACF



Un proceso estocástico de segundo orden $\boldsymbol{X}$ se dice que
es *débilmente estacionario* si la covarianza entre $X_s$ y $X_t$
solo depende de la diferencia $s-t$ para todo $s,t\in\mathbb{Z}$.

En tal caso se denomina *función de <u>auto</u>-covarianzas* de
$\boldsymbol{X}$ a la secuencia
$$\boldsymbol{\gamma}(z)\;=\;(\gamma_k\mid
k\in\mathbb{Z})\;=\;\sum_{-\infty}^{\infty} \gamma_k z^k$$ donde
$\;\gamma_k=Cov(X_{t+k}, X_{t})\;$ para $k\in\mathbb{Z}$.

**Propiedades** de la función de autocovarianzas $\boldsymbol{\gamma}$ (ACF):

-   $\gamma_0\geq0$
-   $\boldsymbol{\gamma}$ <u>es definida positiva</u>; y por tanto,
    -   $\boldsymbol{\gamma}$ es simétrica: $\gamma_k=\gamma_{-k}$
    -   $\boldsymbol{\gamma}$ es acotada: $|\gamma_k|\leq\gamma_0$

llamamos *función de autocorrelación* (ACF) a la
secuencia:
$\;\boldsymbol{\rho}=\frac{1}{\gamma_0}(\boldsymbol{\gamma})
=\sum\limits_{k\in\mathbb{Z}}\frac{\gamma_k}{\gamma_0}z^k$.



## Ejemplos de procesos (débilmente) estacionarios



### Proceso de ruido blanco



Una secuencia $\boldsymbol{U}=(U_t\mid t\in\mathbb{Z})$ de variables
aleatorias incorreladas y tales que $$E(U_t)=0\quad\text{ y }\quad
Var(U_t)=E(U_t^2)=\sigma^2$$ para $\;t\in\mathbb{Z}\;$ y
$\;0<\sigma^2<\infty\;$ se llama *proceso de ruido blanco*.
$\quad\boldsymbol{U}\sim WN(0,\sigma^2)$.



Al ser variables aleatorias incorreladas, su función de
autocovarianzas es $$\boldsymbol{\gamma}(z)\;=\;\sigma^2
z^0\;=\;(\ldots,0,0,\sigma^2,0,0,\ldots)$$

-   Es el proceso estacionario (no trivial) más sencillo.
-   Este proceso es el pilar sobre el que definiremos el resto de
    ejemplos.



### Procesos lineales



Sea $\boldsymbol{U}\sim WN(0,\sigma^2)$ y sea $\boldsymbol{\theta}\in
\ell^2$; una secuencia de <u>cuadrado sumable</u>
$\;\sum\limits_{j\in\mathbb{Z}}\theta_j^2<\infty$.

Denominamos *proceso lineal* al proceso estocástico
$\boldsymbol{X}=\boldsymbol{\theta}*\boldsymbol{U}$ cuyos elementos son $$X_t
\;=\;(\boldsymbol{\theta}*\boldsymbol{U})_t
\;=\;\boldsymbol{\theta}(B)U_t \;=\;\sum_{j=-\infty}^\infty \theta_j
U_{t-j};\qquad t\in\mathbb{Z}.$$



El proceso lineal es *\`\`causal''* si además $\boldsymbol{\theta}$ es
una <u>serie formal</u> (i.e.,
$cogrado(\boldsymbol{\theta})\geq{\color{blue}{0}}$)
$$X_t=\sum_{\color{blue}{j=0}}^\infty \theta_j U_{t-j};\qquad
t\in\mathbb{Z}$$ $\;$ (pues cada $X_t$ es una suma de variables "*del
presente y pasado*").



La clase de procesos lineales incluye muchas e importantes subclases
de procesos, algunas de las cuales son objeto principal de estudio de
este curso.



#### Media móvil infinita. MA($\infty$)



Sea $\;\boldsymbol{U}\sim WN(0,\sigma^2)\;$ y sea
$\;\boldsymbol{\theta}\in \ell^2\;$ una serie formal con <u>infinitos
términos NO nulos</u>; entonces el proceso estocástico
$\boldsymbol{\theta}*\boldsymbol{U}$, cuyos elementos son $$X_t
\;=\;(\boldsymbol{\theta}*\boldsymbol{U})_t
\;=\;\boldsymbol{\theta}(B)U_t \;=\;\sum_{j=0}^\infty \theta_j
U_{t-j};\qquad t\in\mathbb{Z}$$ se denomina proceso de *media móvil
infinita* MA($\infty$).



Algunas clases de procesos lineales tienen una representación
parsimoniosa, pues basta un número finito de parámetros para
representarlos completamente. Por ejemplo, cuando
$\boldsymbol{\theta}$ tiene un número finito de términos no nulos&#x2026;



#### Proceso de media móvil de orden $q$. MA($q$)



Sea $\;\boldsymbol{U}\sim WN(0,\sigma^2)\;$ y sea
$\;\boldsymbol{\theta}\;$ un <u>polinomio de grado $q$</u> con $\theta_0=1$;
entonces el proceso estocástico $\boldsymbol{\theta}*\boldsymbol{U}$,
cuyos elementos son $$X_t \;=\;(\boldsymbol{\theta}*\boldsymbol{U})_t
\;=\;\boldsymbol{\theta}(B)U_t \;=\;\sum_{j=0}^q\theta_j
U_{t-j};\qquad t\in\mathbb{Z}$$ se denomina proceso de *media móvil*
MA($q$).



Es decir:
$$ X_t = U_t + \theta_1 U_{t-1} + \cdots + \theta_q U_{t-q}.$$



Hay otros procesos lineales con representación parsimoniosa.



#### Proceso autorregresivo de orden $p$. AR($p$)



Sea $\;\boldsymbol{U}\sim WN(0,\sigma^2)\;$, se denomina *proceso
autorregresivo de orden $p$* a aquel proceso estocástico estacionario
$\;\boldsymbol{X}\;$ que es la solución de la siguiente ecuación en
diferencias $$\boldsymbol{\phi}*\boldsymbol{X}=\boldsymbol{U}$$ donde
$\;\boldsymbol{\phi}\;$ un <u>polinomio de grado $p$</u> con $\phi_0=1$;



Por tanto, $$(\boldsymbol{\phi}*\boldsymbol{X})_t=
\boldsymbol{\phi}(\mathsf{B})X_t= \sum_{j=0}^p \phi_j X_{t-j} = U_t.$$



Es decir $\boldsymbol{X}=(X_t\mid t\in\mathbb{Z})$ es una solución de
la siguiente ecuación en diferencias: $$X_t + \phi_1 X_{t-1} +
\cdots + \phi_q X_{t-q} = U_t.$$



El problema con la anterior definición es que la solución
$\boldsymbol{X}$ de la ecuación
$\boldsymbol{\phi}*\boldsymbol{X}=\boldsymbol{U}$ no es única. Para
verlo basta despejar $\boldsymbol{X}$.

Multiplicando ambos lados de la igualdad por una inversa de
$\boldsymbol{\phi}$ lo logramos
$$\boldsymbol{X}=inversa(\boldsymbol{\phi})*\boldsymbol{U},$$ si
denotamos $inversa(\boldsymbol{\phi})$ con $\boldsymbol{a}$ entonces
$$X_t=\boldsymbol{a}(\mathsf{B})U_t=\sum_{j\in\mathbb{Z}} a_j
U_{t-j}.$$



Pero&#x2026; ¿Qué secuencia $\boldsymbol{a}$ usamos como inversa de
$\boldsymbol{\phi}$? Recuerde que hay infinitas y la mayoría no son
sumables (si el polinomio $\boldsymbol{\phi}$ tiene raíces unitarias
ninguna lo es).

<div class="org-center">
<p>
En tal caso la expresión
$\;\boldsymbol{a}(\mathsf{B})U_t=\sum\limits_{j=-\infty}^\infty a_j
U_{t-j}\;$ carece de sentido (no converge).
</p>
</div>



**Requisitos** sobre el polinomio autorregresivo $\boldsymbol{\phi}$

1.  Para tener un <u>proceso lineal</u>, exigiremos que $\boldsymbol{\phi}$
    <u>no tenga raíces de módulo 1</u>.
    
    Entonces existe la inversa $\boldsymbol{\phi}^{-1} \in
       \ell^1\subset\ell^2$ (la abolutamente sumable).
    
    La inversa $\boldsymbol{a}=\boldsymbol{\phi}^{-1}$ corresponde a la
    única solución *estacionaria* de
    $\boldsymbol{\phi}*\boldsymbol{X}=\boldsymbol{U}$.  (Si
    $\boldsymbol{\phi}$ tuviera raíces de módulo 1 no existiría ni
    $\boldsymbol{\phi}^{-1}$ ni solución estacionaria).

$$X_t=\boldsymbol{\phi}^{-1}(\mathsf{B})U_t=\sum_{j=-\infty}^\infty
a_j U_{t-j}$$

1.  Para tener un <u>proceso lineal causal</u> exigiremos que las raíces de
    $\boldsymbol{\phi}$ sean mayores que 1 en valor absoluto (<u>raíces
    fuera del círculo unidad</u>).

$$X_t=\boldsymbol{\phi}^{-1}(\mathsf{B})U_t=\sum_{j=0}^\infty a_j
U_{t-j}$$



El siguiente modelo lineal es una combinación (o generalización) de
los dos anteriores.



#### Proceso Autorregresivo de media móvil. ARMA($p,q$)



Sea $\;\boldsymbol{U}\sim WN(0,\sigma^2)\;$, se denomina *proceso
autorregresivo de media móvil $(p,q)$* al proceso estocástico
estacionario $\;\boldsymbol{X}\;$ que es la solución de la ecuación en
diferencias:
$$\boldsymbol{\phi}*\boldsymbol{X}=\boldsymbol{\theta}*\boldsymbol{U}$$
donde el polinomio *autorregresivo* $\;\boldsymbol{\phi}\;$ tiene
<u>grado $p$</u> con $\phi_0=1$ y con todas sus raíces fuera del círculo
unidad (*por los motivos anteriormente vistos*); y el polinomio *de
media móvil* $\;\boldsymbol{\theta}\;$ es <u>de grado $q$</u> con
$\theta_0=1$;

$$\text{Es decir}\quad
\boldsymbol{X}=\frac{\boldsymbol{\theta}}{\boldsymbol{\phi}}*\boldsymbol{U}
\quad\text{donde}\quad
\frac{1}{\boldsymbol{\phi}}\equiv\boldsymbol{\phi}^{-1}\in\ell^1$$



Tanto $\boldsymbol{\phi}^{-1}$ como $\boldsymbol{\theta}$ son
absolutamente sumables y como $\ell^1$ es un anillo,
$\boldsymbol{\phi}^{-1}*\boldsymbol{\theta}\equiv\frac{\boldsymbol{\theta}}{\boldsymbol{\phi}}\in\ell^1$
(también es absolutamente sumable y por tanto de cuadrado sumable),
consecuentemente el proceso estocástico es un proceso lineal.
$$X_t=\frac{\boldsymbol{\theta}}{\boldsymbol{\phi}}(\mathsf{B})U_t=\sum_{j=0}^\infty
a_j U_{t-j}$$ donde
$\boldsymbol{a}=\boldsymbol{\phi}^{-1}*\boldsymbol{\theta}$.

